In [15]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [17]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

In [21]:
page_url = "https://www.youtube.com/watch?v=kffacxfA7G4"
driver.get(page_url)

In [22]:
driver.maximize_window()
time.sleep(15)
print("Ready To Go")

Ready To Go


In [23]:
prev_h = 0
while True:
    height = driver.execute_script("""
            function getActualHeight() {
                return Math.max(
                    Math.max(document.body.scrollHeight, document.documentElement.scrollHeight),
                    Math.max(document.body.offsetHeight, document.documentElement.offsetHeight),
                    Math.max(document.body.clientHeight, document.documentElement.clientHeight)
                );
            }
            return getActualHeight();
        """)
    driver.execute_script(f"window.scrollTo({prev_h},{prev_h + 200})")
    time.sleep(1)
    prev_h +=200  
    
    if prev_h >= height:
        break
    if height >= 100000:
        break

In [24]:
content = driver.find_elements(By.CLASS_NAME, 'style-scope ytd-comment-renderer')

comments = []
for i in range(len(content)):
    comments.append(((content[i].text).split('\n'))[2])
    if i == 100:
        break

In [7]:
driver.close()

In [33]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [34]:
data = pd.read_csv('numbered_dataset_words.csv',usecols= ['sl.no','score','word'])

In [35]:
words_dict = {}
scores_dict = {}
for s,k,v in zip(data['sl.no'],data['word'],data['score']):
    words_dict[s] = k
    scores_dict[s] = v

In [36]:
def keymaker(filtered_sentence):
    keystore=[]
    for i in filtered_sentence:
        if i in words_dict.values():
            for key, value in words_dict.items():
                if i == value:
                    keystore.append(key)
    return keystore

In [37]:
def polarity(filtered_sentence):
    polarity=[]
    for i in keymaker(filtered_sentence):
        polarity.append(scores_dict.get(i))
    total=0
    for j in polarity:
        j=int(j)
        total=total + j
    return int(total)

In [38]:
polarity_list = []
count = 0
for i in comments:
    stop_words = set(stopwords.words('english'))
    wtokens = word_tokenize(i)
    fil_sen = []
    for w in wtokens:
        if w not in stop_words:
            fil_sen.append(w)
    count +=1 
    if count == 100:
        break
            
    p=polarity(fil_sen)
    polarity_list.append(p)

In [39]:
polarity_list

net_polarity = 0

for i in polarity_list: 
    net_polarity += i

In [40]:
def senti_analyse(net_polarity):
    if net_polarity>=1:
        print("POSITIVE")
    elif net_polarity<0:
        print("NEGATIVE")
    else:
        print("NEUTRAL")

In [41]:
senti_analyse(net_polarity)

NEGATIVE
